reference: https://towardsdatascience.com/digit-classification-with-single-layer-perceptron-9a4e7d4d9628

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# K-Fold Cross-Validation
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = ['accuracy', 'precision', 'recall', 'f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

In [2]:
def activation_function(prediction):
    """
    Receives the output of the perceptron's function as parameter, and applies the
    activation function on it.
    For the purpose of this project, the activation function maps the negative outputs
    to 0 and the positive ones to 1
    """
    if prediction >= 0:
        return 1
    return 0
  

def predict(x, weights, bias):
    """
    Predicts the class of a given data point (x), by applying the Perceptron's 
    function, and the activation function lastly.
    As both weights and x are vectors, the dot product is used.
    """
    prediction = np.dot(weights, x) + bias
    prediction = activation_function(prediction)

    
    
    return prediction

In [3]:
def forward_propagation(x, y, weights, bias): 
    """
    x: training data as a vector (nparray), where each value corresponds
        to a feature's value
    y: label (0 or 1)
    weights: weights of the perceptron
    bias: bias
    """
    y_pred = predict(x, weights, bias)  
    
    if y == "Letter":
      y=0
    else:
      y=1

    loss = (y_pred - y)**2 
    d_loss = 2*(y_pred - y)
    
    return y_pred, loss, d_loss

def backpropagation(x, d_loss):
    """
    Performs the Backpropagation step on a given data point.
    receives as input the data point, the Perceptron's weights and the partial derivative of the loss
    over the predicted y.
    The received derivative is used to calculate the partial derivative of the loss over the weight of each feature.
    A list with the partial derivatives of the loss over each weight is returned.
    """
    partial_derivates = list()
    for feature_value in x:
        partial_derivates.append(d_loss*feature_value)
        
    return partial_derivates   

In [25]:
import matplotlib.pyplot as plt
import numpy as np

def optimize_perceptron(x, y, learning_rate):
    """
    Optimizes the Perceptron's weights by looping over the same steps for as many epochs as the user wants.
    Steps:
    1. Forward propagate data point
    2. Backpropagate
    3. Update weights
    4. Check stop conditions while looping
    
    It is worth noting that a history of the Perceptron's losses over each epoch is kept,
    which will be used
    """
    epoch = 0
    error = 999
    weights = np.random.rand(x.shape[1])
    bias = np.random.rand()
    
    errors = list()
    epochs = list()
    accuracy = list()
    
    # Loop until stop conditions are met
    while (epoch <= 250) : #and (error > 9e-4)
        
        loss_ = 0
        # Loop over every data point
        for i in range(x.shape[0]):
            
            # Forward Propagation on each data point
            y_pred, loss, d_loss = forward_propagation(x[i], y[i], weights, bias)

            # Backpropagation
            partial_derivates = backpropagation(x[i], d_loss)
            
            # Learn by updating the weights of the perceptron
            weights = weights - (learning_rate * np.array(partial_derivates))

        # Evaluate the results
        for index, feature_value_test in enumerate(x):
            y_pred, loss, d_loss = forward_propagation(feature_value_test, y[index], weights, bias)
            loss_ += loss

        errors.append(loss_/len(x))
        epochs.append(epoch)
        error = errors[-1]
        epoch += 1
        


        xpoints = epochs
        ypoints = errors

        plt.plot(xpoints, ypoints)

        plt.xlabel('epochs')
        # naming the y axis
        plt.ylabel('loss')
          
        # giving a title to my graph
        plt.title('Loss Graph')
        plt.show()

        accuracy.append(calculate_accuracy(x_test, y_test, weights, bias))

        xpoints = epochs
        ypoints = accuracy

        plt.plot(xpoints, ypoints)

        plt.xlabel('accuracy')
        # naming the y axis
        plt.ylabel('loss')
          
        # giving a title to my graph
        plt.title('Accuracy Graph')
        plt.show()


        print('Epoch {}. loss: {}'.format(epoch, errors[-1]))

    
    return weights, bias, errors

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# data = pd.read_csv('/content/drive/MyDrive/LAU/Fall-2022/IEA/Projects/Project 1/csv_ensemble2/csv_ensemble2/classes.csv')
data = pd.read_csv('/content/drive/MyDrive/LAU/Fall-2022/IEA/Projects/Project 1/csv_ensemble1/csv_ensemble1/training_set_upper_lower.csv')

# # Take only data with labels 1
# data_ones = data[data['label'] == 1]

# # Take only data with labels 0
# data_zeros = data[data['label'] == 0]

# # Concatenate instances with label 0 and 1
# data = pd.concat([data_ones, data_zeros])
# print(data.shape)
# print(np.unique(data['label'].to_numpy()))


# Split dataset with 75% training data and 25% test data
train_data, test_data = train_test_split(data, test_size=0.25, random_state=1, shuffle=True)

# Split datasets into features and labels
x_train = train_data.drop('0', axis=1).to_numpy()
x_test = test_data.drop('0', axis=1).to_numpy()
y_train = train_data['0'].to_numpy()
y_test = test_data['0'].to_numpy()


print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print('Values before rescaling: ', np.unique(x_train))

#Feature Scaling
scaler = MinMaxScaler()
x_train =scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

print('Values after rescaling: ', np.unique(x_train))

(1719, 170)
(573, 170)
(1719,)
(573,)
Values before rescaling:  [    0     1     2 ... 25498 25499 25500]
Values after rescaling:  [0.00000000e+00 3.92156863e-05 7.84313725e-05 ... 9.99960784e-01
 1.00000000e+00 1.00000000e+00]


In [6]:
def flatten_image(image):
    """
    Receives an image as a matrix and converts it to 
    a vector
    """
    image.resize((image.shape[0]*image.shape[1],))

In [24]:
weights, bias, errors = optimize_perceptron(x_train, y_train,learning_rate = 0.001)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
def calculate_accuracy(x_test, y_test, weights, bias):
    
    # Initialize True Positive, True Negative, False Positive and False Negative
    tp, tn, fp, fn = 0, 0, 0, 0

    for sample, label in zip(x_test, y_test):


        prediction = predict(sample, weights, bias)
        
        if label == "Letter":
          label=0
        else:
          label=1

        if prediction == label:
            if prediction == 1:
                tp += 1
            else:
                tn += 1
        else:
            if prediction == 1:
                fp += 1
            else:
                fn += 1

    accuracy = (tp + tn)/(tp + tn + fp + fn)
    return accuracy

In [11]:
print(calculate_accuracy(x_test,y_test,weights,bias))

0.9838945827232797


In [12]:
print(epochs)

NameError: ignored